In [ ]:
import pylab as plt
import tqdm as tqdm
import pandas as pd
from herbie import FastHerbie
import cartopy.crs as ccrs
import numpy as np
import xarray as xr
import warnings
warnings.filterwarnings('ignore')
import zarr
from IPython.utils import io

In [ ]:
#Specify the time range over which to generate zarr files
#DATES = pd.date_range('2021-03-10 15:00', '2021-03-10 20:00', freq='1H')
DATES = pd.date_range('2021-03-13 00:00', '2021-03-14 20:00', freq='1H')

In [ ]:
# Create zarr filename prefix
of_name = DATES[0].strftime("%y-%m-%d-%Hz") + '_to_' + DATES[-1].strftime("%y-%m-%d-%Hz")

In [ ]:
# Instantiate herbie classes
H = FastHerbie(DATES, model="hrrr", product="sfc", save_dir='/Users/pres026/Research/Datasets/hrrr_data', fxx=[1])
Hnat = FastHerbie(DATES, model="hrrr", product="nat", save_dir='/Users/pres026/Research/Datasets/hrrr_data', fxx=[1])

In [ ]:
# Define surface variables to be read and download them if needed
surface_name = {} 
surface_name['VIL'] = "VIL"
surface_name['SST'] = ":TMP:surface"  
surface_name['T2m'] ="TMP:2 m"
surface_name['QV2m'] ="SPFH:2 m"
surface_name['U80'] = "UGRD:80 m"
surface_name['V80'] = "VGRD:80 m"
surface_name['U10'] = "UGRD:10 m"
surface_name['V10'] = "VGRD:10 m"
surface_name['PSFC'] = "PRES:surface"
surface_name['LHTFL'] = "LHTFL:surface"
surface_name['SHTFL'] = "SHTFL:surface"
surface_name['DSWRF'] = "DSWRF:surface"
surface_name['LCDC'] = "LCDC"
surface_name['WIND'] = "WIND"
surface_name['HPBL'] = "HPBL"
surface_name['LAND'] = "LAND"
surface_name['DLWRF'] = "DLWRF:surface"
for name, regex in tqdm.tqdm(surface_name.items()):
    H.download(regex)
    

In [ ]:
# Convert downloaded grib2 files to zarr
for name, regex in tqdm.tqdm(surface_name.items()):
    with io.capture_output() as captured:
        ds = H.xarray(regex, max_threads=1)
    ds.to_zarr(of_name + '_surface.zarr', mode='a')
    ds.close()

In [ ]:
# Define native model level fields to be downloaded
level_name = {} 
level_name["U"] = ":(?:U)GRD:[0-9]+ hybrid"
level_name["V"] = ":(?:V)GRD:[0-9]+ hybrid"
level_name["P"] = ":(?:PRES):[0-9]+ hybrid"
level_name["T"] = ":(?:TMP):[0-9]+ hybrid"
level_name["QV"] = ":(?:SPFH):[0-9]+ hybrid"
level_name["QC"] = ":(?:CLMR):[0-9]+ hybrid"
level_name["QI"] = ":(?:CIMIXR):[0-9]+ hybrid"
level_name["Z"] =":(?:HGT:)[0-9]+ hybrid"

for name, regex in tqdm.tqdm(level_name.items()):
    with io.capture_output() as captured:
            Hnat.xarray(regex, max_threads=1).to_zarr(of_name + '_levels.zarr', mode='a')